<a href="https://colab.research.google.com/github/theAkhileshRai/Projects/blob/master/TestPred_Leak_Pred_DNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('/content/New_Normalised_skmin_max.csv',index_col=False)

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,108025,9783,2019-02-27 14:30:00,2.0,0.166421,0.589277,0.714105,0.795629,0.230168,0.166421,0.343941,0.797193,0.609704,0.230168,Saltlake_K71010B,1.0
1,1,108026,9784,2019-02-27 15:30:00,2.0,0.169243,0.591297,0.711149,0.793577,0.248063,0.169243,0.345498,0.795464,0.608471,0.248063,Saltlake_K71010B,1.0
2,2,108027,9785,2019-02-27 16:30:00,2.0,0.178583,0.596431,0.708357,0.791206,0.263139,0.178583,0.350036,0.794214,0.607237,0.263139,Saltlake_K71010B,1.0
3,3,108028,9786,2019-02-27 17:30:00,2.0,0.184180,0.602012,0.698249,0.785171,0.308088,0.184180,0.354039,0.787344,0.603296,0.308088,Saltlake_K71010B,1.0
4,4,108029,9787,2019-02-27 18:30:00,2.0,0.179204,0.574376,0.754675,0.802939,0.284134,0.179204,0.338874,0.844417,0.625370,0.284134,Saltlake_K71010B,1.0


In [ ]:
# df['s_temp_mean*s_pressure_mean'] = df['s_pressure_mean']*df['s_temp_mean']
# df['d_temp_mean*d_pressure_mean'] = df['d_pressure_mean']*df['d_temp_mean']

In [ ]:
# df['s_temp_crank*s_pressure_crank'] = df['s_temp_crank']*df['s_pressure_crank']
# df['d_temp_crank*d_pressure_crank'] = df['d_pressure_crank']*df['d_temp_crank']

In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,108025,9783,2019-02-27 14:30:00,2.0,0.166421,0.589277,0.714105,0.795629,0.230168,0.166421,0.343941,0.797193,0.609704,0.230168,Saltlake_K71010B,1.0
1,1,108026,9784,2019-02-27 15:30:00,2.0,0.169243,0.591297,0.711149,0.793577,0.248063,0.169243,0.345498,0.795464,0.608471,0.248063,Saltlake_K71010B,1.0
2,2,108027,9785,2019-02-27 16:30:00,2.0,0.178583,0.596431,0.708357,0.791206,0.263139,0.178583,0.350036,0.794214,0.607237,0.263139,Saltlake_K71010B,1.0
3,3,108028,9786,2019-02-27 17:30:00,2.0,0.184180,0.602012,0.698249,0.785171,0.308088,0.184180,0.354039,0.787344,0.603296,0.308088,Saltlake_K71010B,1.0
4,4,108029,9787,2019-02-27 18:30:00,2.0,0.179204,0.574376,0.754675,0.802939,0.284134,0.179204,0.338874,0.844417,0.625370,0.284134,Saltlake_K71010B,1.0


In [ ]:
# df1 = pd.read_csv('/content/old_normalised_assets-2.csv',index_col=False)
# df2 = pd.read_csv('/content/new_normalised_assets-2.csv',index_col=False)

In [ ]:
# 'Clark county_6', 'RJ booster_1', 'Strawberry_3.csv'

In [6]:
df['Asset'].unique()   

array(['Saltlake_K71010B', 'Lightning_Unit4', 'Reagon_Unit3',
       'West Turkey_Unit2', 'Strawberry_Unit1', 'Reagon_Unit6'],
      dtype=object)

In [7]:
len(df['Asset'].unique() )

6

In [ ]:
# 'Clark county_6') | ('La Porte_K11A') |\
#              ('Strawberry_Unit1') | ('RJ booster_1') | ('El Dorado_Unit'

In [8]:
df['Binary'] = df['Binary'].fillna(0)
df = df.fillna(0)
#df1['multi_class'] = df1['multi_class'].fillna(0)

In [9]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,108025,9783,2019-02-27 14:30:00,2.0,0.166421,0.589277,0.714105,0.795629,0.230168,0.166421,0.343941,0.797193,0.609704,0.230168,Saltlake_K71010B,1.0
1,1,108026,9784,2019-02-27 15:30:00,2.0,0.169243,0.591297,0.711149,0.793577,0.248063,0.169243,0.345498,0.795464,0.608471,0.248063,Saltlake_K71010B,1.0
2,2,108027,9785,2019-02-27 16:30:00,2.0,0.178583,0.596431,0.708357,0.791206,0.263139,0.178583,0.350036,0.794214,0.607237,0.263139,Saltlake_K71010B,1.0
3,3,108028,9786,2019-02-27 17:30:00,2.0,0.184180,0.602012,0.698249,0.785171,0.308088,0.184180,0.354039,0.787344,0.603296,0.308088,Saltlake_K71010B,1.0
4,4,108029,9787,2019-02-27 18:30:00,2.0,0.179204,0.574376,0.754675,0.802939,0.284134,0.179204,0.338874,0.844417,0.625370,0.284134,Saltlake_K71010B,1.0


In [10]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'time', 'cyl',
       's_temp_crank', 'd_temp_crank', 's_pressure_crank', 'd_pressure_crank',
       'rpm_crank', 's_temp_mean', 'd_temp_mean', 's_pressure_mean',
       'd_pressure_mean', 'rpm_mean', 'Asset', 'Binary'],
      dtype='object')

In [11]:
X_columns = [ 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',\
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',\
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean']

In [12]:
len(X_columns)

11

In [14]:
X = df[X_columns]

In [15]:
X.shape

(31336, 11)

# build  with mean

In [18]:
y_true = df.iloc[:,16]

In [19]:
y_true.unique()

array([1., 0.])

In [21]:
from collections import Counter

In [27]:
Counter(y_true)

Counter({0.0: 9475, 1.0: 21861})

64->16->4->1

In [28]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 11.
        self.layer_1 = nn.Linear(11, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 64)
        self.layer_4 = nn.Linear(64, 10)
        self.layer_5 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        xavier_uniform_(self.layer_out.weight)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(10)
        self.sig = nn.Sigmoid()

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        x = self.sig(x)
        
        return x

In [29]:
EPOCHS = 10
BATCH_SIZE = 75
LEARNING_RATE = 0.0792

In [46]:
## train data
# class trainData(Dataset):
    
#     def __init__(self, X_data, y_data):
#         self.X_data = X_data
#         self.y_data = y_data
        
#     def __getitem__(self, index):
#         return self.X_data[index], self.y_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

# y_train = y_train.values
# train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
# ## val data    
# class valData(Dataset):
    
#     def __init__(self, X_data):
#         self.X_data = X_data
        
#     def __getitem__(self, index):
#         return self.X_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

test_data = testData(torch.FloatTensor(X.values))
# val_data = valData(torch.FloatTensor(X_val))

In [47]:
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset=val_data, batch_size=1)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [49]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=11, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sig): Sigmoid()
)

In [50]:
import pickle
filename = 'binarynn_dcp_model.pkl'

model.load_state_dict(torch.load(filename))

<All keys matched successfully>

In [51]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [52]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [53]:
y_pred_list = []
pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        pred_list.append(y_test_pred.cpu().numpy())
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [54]:
test_data = pd.DataFrame({'Values': y_pred_list, 'Predictions': pred_list})

In [58]:
test_data.head()

,Values,Predictions
0,0.0,[[6.382671e-05]]
1,0.0,[[5.9588452e-05]]
2,0.0,[[5.750307e-05]]
3,0.0,[[4.6580084e-05]]
4,0.0,[[7.209934e-05]]


In [55]:
test_data.to_csv('Test Predictions.csv')

In [60]:
confusion_matrix(y_true, y_pred_list)

array([[ 9457,    18],
       [21860,     1]])

# New Assets

In [62]:
print(classification_report(y_true, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.30      1.00      0.46      9475
         1.0       0.05      0.00      0.00     21861

    accuracy                           0.30     31336
   macro avg       0.18      0.50      0.23     31336
weighted avg       0.13      0.30      0.14     31336



In [ ]:
model.load_state_dict(torch.load(filename))

<All keys matched successfully>